# Humane Bench QA Analysis

This notebook provides comprehensive quality assurance and data science analysis for the Humane Bench dataset.

**Contents:**
1. Exploratory Data Analysis (EDA)
2. Distribution Analysis & Visualization
3. Stratified Random Sampling for Manual Review
4. Inter-Rater Reliability Analysis (Fleiss' Kappa)

**Usage:**
- Run cells sequentially for full analysis
- Section 3 exports a sample CSV for manual review
- Section 4 analyzes inter-rater agreement after reviews are completed

In [ ]:
# Import required libraries
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from pathlib import Path
from collections import Counter
from typing import List, Dict

# Add parent directory to path for imports
sys.path.insert(0, str(Path.cwd().parent))

# Import constants from config
from config import HUMANE_PRINCIPLES, TOPIC_DOMAINS, VULNERABLE_POPULATIONS, DATASET_PATH

# Set visualization style
sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print("✓ Libraries loaded successfully")

## 1. Load and Explore Dataset

In [ ]:
# Load dataset
def load_dataset(path: str = DATASET_PATH) -> pd.DataFrame:
    """Load JSONL dataset into a pandas DataFrame."""
    data = []
    with open(path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    
    df = pd.DataFrame(data)
    
    # Extract metadata fields into columns
    df['principle'] = df['metadata'].apply(lambda x: x.get('principle', ''))
    df['domain'] = df['metadata'].apply(lambda x: x.get('domain', ''))
    df['vulnerable_population'] = df['metadata'].apply(lambda x: x.get('vulnerable-population', ''))
    
    # Calculate input length
    df['input_length'] = df['input'].str.len()
    
    return df

# Load data
df = load_dataset()

print(f"✓ Loaded {len(df)} scenarios")
print(f"\nDataset shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")

In [ ]:
# Display first few rows
print("Sample scenarios:")
df[['id', 'input', 'principle', 'domain', 'vulnerable_population']].head(10)

In [ ]:
# Basic statistics
print("=== DATASET OVERVIEW ===")
print(f"Total scenarios: {len(df)}")
print(f"Unique principles: {df['principle'].nunique()}")
print(f"Unique domains: {df['domain'].nunique()}")
print(f"Scenarios with vulnerable populations: {df['vulnerable_population'].astype(bool).sum()} ({df['vulnerable_population'].astype(bool).sum()/len(df)*100:.1f}%)")
print(f"\nInput length: mean={df['input_length'].mean():.1f}, median={df['input_length'].median():.1f}, min={df['input_length'].min()}, max={df['input_length'].max()}")

## 2. Distribution Analysis & Visualization

In [ ]:
# Principle distribution
principle_counts = df['principle'].value_counts()
principle_pct = df['principle'].value_counts(normalize=True) * 100

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Count plot
principle_counts.plot(kind='barh', ax=ax1, color='steelblue')
ax1.set_title('Scenarios per Principle (Count)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Count')
ax1.set_ylabel('Principle')
ax1.axvline(len(df) / len(HUMANE_PRINCIPLES), color='red', linestyle='--', label=f'Target (balanced): {len(df) / len(HUMANE_PRINCIPLES):.0f}')
ax1.legend()

# Percentage plot
principle_pct.plot(kind='barh', ax=ax2, color='coral')
ax2.set_title('Scenarios per Principle (Percentage)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Percentage (%)')
ax2.set_ylabel('')
ax2.axvline(100 / len(HUMANE_PRINCIPLES), color='red', linestyle='--', label=f'Target: {100/len(HUMANE_PRINCIPLES):.1f}%')
ax2.legend()

plt.tight_layout()
plt.show()

# Print detailed breakdown
print("\n=== PRINCIPLE DISTRIBUTION ===")
target_pct = 100.0 / len(HUMANE_PRINCIPLES)
for principle in HUMANE_PRINCIPLES:
    count = principle_counts.get(principle, 0)
    pct = principle_pct.get(principle, 0.0)
    deviation = abs(pct - target_pct)
    status = "✓" if deviation < 5.0 else "⚠"
    print(f"{status} {principle:40s} {count:4d} ({pct:5.1f}%) [deviation: {deviation:4.1f}%]")

In [ ]:
# Domain distribution
domain_counts = df['domain'].value_counts().head(15)

plt.figure(figsize=(12, 8))
domain_counts.plot(kind='barh', color='mediumseagreen')
plt.title('Top 15 Domains (Count)', fontsize=14, fontweight='bold')
plt.xlabel('Count')
plt.ylabel('Domain')
plt.tight_layout()
plt.show()

print(f"\n=== DOMAIN DISTRIBUTION ===")
print(f"Total unique domains: {df['domain'].nunique()}")
print(f"\nTop 15 domains:")
for domain, count in domain_counts.items():
    pct = count / len(df) * 100
    in_standard = "✓" if domain in TOPIC_DOMAINS else "⚠"
    print(f"{in_standard} {domain:30s} {count:4d} ({pct:5.1f}%)")

In [ ]:
# Vulnerable population distribution
vuln_pop_counts = df[df['vulnerable_population'] != '']['vulnerable_population'].value_counts()

if len(vuln_pop_counts) > 0:
    plt.figure(figsize=(10, 6))
    vuln_pop_counts.plot(kind='bar', color='indianred')
    plt.title('Vulnerable Population Distribution', fontsize=14, fontweight='bold')
    plt.xlabel('Population Type')
    plt.ylabel('Count')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

total_with_vuln = (df['vulnerable_population'] != '').sum()
total_without_vuln = len(df) - total_with_vuln

print(f"\n=== VULNERABLE POPULATION DISTRIBUTION ===")
print(f"With vulnerable population: {total_with_vuln} ({total_with_vuln/len(df)*100:.1f}%)")
print(f"Without: {total_without_vuln} ({total_without_vuln/len(df)*100:.1f}%)")
print(f"\nBreakdown:")
for pop, count in vuln_pop_counts.items():
    pct = count / len(df) * 100
    print(f"  {pop:25s} {count:4d} ({pct:5.1f}%)")

In [ ]:
# Heatmap: Principle x Domain coverage
crosstab = pd.crosstab(df['principle'], df['domain'])

# Show top domains only for readability
top_domains = df['domain'].value_counts().head(12).index
crosstab_subset = crosstab[top_domains]

plt.figure(figsize=(14, 8))
sns.heatmap(crosstab_subset, annot=True, fmt='d', cmap='YlOrRd', cbar_kws={'label': 'Count'})
plt.title('Principle × Domain Coverage (Top 12 Domains)', fontsize=14, fontweight='bold')
plt.xlabel('Domain')
plt.ylabel('Principle')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Input length distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Histogram
ax1.hist(df['input_length'], bins=50, color='skyblue', edgecolor='black')
ax1.axvline(df['input_length'].mean(), color='red', linestyle='--', label=f'Mean: {df["input_length"].mean():.1f}')
ax1.axvline(df['input_length'].median(), color='green', linestyle='--', label=f'Median: {df["input_length"].median():.1f}')
ax1.set_title('Input Length Distribution', fontsize=14, fontweight='bold')
ax1.set_xlabel('Character Count')
ax1.set_ylabel('Frequency')
ax1.legend()

# Box plot by principle
df.boxplot(column='input_length', by='principle', ax=ax2, rot=90)
ax2.set_title('Input Length by Principle', fontsize=14, fontweight='bold')
ax2.set_xlabel('Principle')
ax2.set_ylabel('Character Count')
plt.suptitle('')  # Remove default title

plt.tight_layout()
plt.show()

print("\n=== INPUT LENGTH STATISTICS ===")
print(df['input_length'].describe())

## 2.5. Mode Collapse Detection - Within-Principle Similarity

Following the DarkBench approach, this section checks for **high cosine similarity within each principle** to ensure heterogeneity and avoid mode collapse.

**What we're checking:**
- Cosine similarity between scenario pairs **within each principle** (not across principles)
- This ensures that scenarios for each principle are sufficiently diverse
- Prevents mode collapse where models give similar responses to all scenarios in a category

**Why this matters:**
- If scenarios within a principle are too similar, models may learn to respond uniformly
- High within-category similarity indicates potential for mode collapse
- DarkBench uses this same approach to validate benchmark quality

**Methodology:**
- Sample ~70 scenarios per principle (or all if fewer)
- Generate embeddings using `text-embedding-3-large`
- Compute all pairwise cosine similarities within each principle
- Flag pairs with similarity ≥0.60 as concerning

**Threshold:** Similarities ≥0.60 indicate scenarios that may be too similar and could lead to mode collapse.

In [ ]:
# Setup and embed scenarios for mode collapse detection
import os
from openai import OpenAI
from sklearn.metrics.pairwise import cosine_similarity
import time
from dotenv import load_dotenv

# Load environment variables
dotenv_path = Path.cwd() / '.env'
load_dotenv(dotenv_path)

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def get_embeddings(texts: List[str], model: str = "text-embedding-3-large", batch_size: int = 100) -> np.ndarray:
    """
    Get embeddings for texts using OpenAI's embedding API.
    
    Args:
        texts: List of text strings to embed
        model: Embedding model to use
        batch_size: Number of texts to process at once
    
    Returns:
        numpy array of embeddings (n_texts, embedding_dim)
    """
    all_embeddings = []
    
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        try:
            response = client.embeddings.create(
                model=model,
                input=batch
            )
            embeddings = [item.embedding for item in response.data]
            all_embeddings.extend(embeddings)
            
            # Rate limiting
            if i + batch_size < len(texts):
                time.sleep(0.5)
                
        except Exception as e:
            print(f"Error processing batch {i//batch_size + 1}: {e}")
            raise
    
    return np.array(all_embeddings)

print("✓ OpenAI client initialized")

# Sample scenarios for each principle
SCENARIOS_PER_PRINCIPLE = 100

print(f"\n=== SAMPLING SCENARIOS ===")
scenario_texts = []
scenario_principles = []

for principle in HUMANE_PRINCIPLES:
    principle_df = df[df['principle'] == principle]
    n_samples = min(SCENARIOS_PER_PRINCIPLE, len(principle_df))
    sample = principle_df.sample(n=n_samples, random_state=42)
    
    scenario_texts.extend(sample['input'].tolist())
    scenario_principles.extend([principle] * n_samples)
    
    print(f"  {principle}: {n_samples} scenarios")

print(f"\nTotal scenarios: {len(scenario_texts)}")

# Generate embeddings
print(f"\n=== GENERATING EMBEDDINGS ===")
print("This may take a minute...")

scenario_embeddings = get_embeddings(scenario_texts)

print(f"✓ Generated {len(scenario_embeddings)} embeddings (dimension: {scenario_embeddings.shape[1]})")

In [ ]:
# Compute within-principle pairwise similarities
print("=== COMPUTING WITHIN-PRINCIPLE SIMILARITIES ===\n")

HIGH_SIM_THRESHOLD = 0.40

within_principle_results = {}

for principle in HUMANE_PRINCIPLES:
    print(f"Processing {principle}...")
    
    # Get scenarios and embeddings for this principle
    principle_mask = np.array(scenario_principles) == principle
    principle_embeddings = scenario_embeddings[principle_mask]
    principle_texts = [scenario_texts[i] for i in range(len(scenario_texts)) if scenario_principles[i] == principle]
    
    n_scenarios = len(principle_texts)
    
    if n_scenarios < 2:
        print(f"  Skipping (only {n_scenarios} scenario)")
        continue
    
    # Compute pairwise similarities
    sim_matrix = cosine_similarity(principle_embeddings)
    
    # Extract upper triangle (excluding diagonal)
    similarities = []
    high_sim_pairs = []
    all_pairs = []
    
    for i in range(n_scenarios):
        for j in range(i + 1, n_scenarios):
            sim = sim_matrix[i, j]
            similarities.append(sim)
            pair_info = (i, j, sim, principle_texts[i], principle_texts[j])
            all_pairs.append(pair_info)
            
            if sim >= HIGH_SIM_THRESHOLD:
                high_sim_pairs.append(pair_info)
    
    # Sort pairs by similarity
    all_pairs.sort(key=lambda x: x[2], reverse=True)
    high_sim_pairs.sort(key=lambda x: x[2], reverse=True)
    
    # Get lowest similarity pairs (most different)
    low_sim_pairs = sorted(all_pairs, key=lambda x: x[2])[:5]
    
    # Store results
    similarities = np.array(similarities)
    within_principle_results[principle] = {
        'n_scenarios': n_scenarios,
        'n_pairs': len(similarities),
        'similarities': similarities,
        'min': similarities.min(),
        'max': similarities.max(),
        'mean': similarities.mean(),
        'std': similarities.std(),
        'n_high_sim': len(high_sim_pairs),
        'high_sim_pairs': high_sim_pairs,
        'low_sim_pairs': low_sim_pairs
    }
    
    print(f"  {n_scenarios} scenarios, {len(similarities)} pairs, {len(high_sim_pairs)} high-similarity (≥{HIGH_SIM_THRESHOLD})")

print(f"\n✓ Computed similarities for all {len(within_principle_results)} principles")

In [ ]:
# Display results per principle
print("=== WITHIN-PRINCIPLE SIMILARITY RESULTS ===\n")

for principle in HUMANE_PRINCIPLES:
    if principle not in within_principle_results:
        continue
    
    results = within_principle_results[principle]
    
    print(f"\n{'='*80}")
    print(f"PRINCIPLE: {principle}")
    print(f"{'='*80}")
    
    print(f"\nStatistics:")
    print(f"  Scenarios analyzed: {results['n_scenarios']}")
    print(f"  Total pairwise comparisons: {results['n_pairs']}")
    print(f"  Min similarity: {results['min']:.4f}")
    print(f"  Mean similarity: {results['mean']:.4f}")
    print(f"  Max similarity: {results['max']:.4f}")
    print(f"  Std deviation: {results['std']:.4f}")
    print(f"  High similarity pairs (≥0.60): {results['n_high_sim']} ({results['n_high_sim']/results['n_pairs']*100:.1f}%)")
    
    # Show high similarity pairs
    if results['n_high_sim'] > 0:
        print(f"\n⚠️  TOP 5 MOST SIMILAR PAIRS (HIGH CONCERN):")
        for i, (idx1, idx2, sim, text1, text2) in enumerate(results['high_sim_pairs'][:5], 1):
            print(f"\n  {i}. Similarity: {sim:.4f}")
            print(f"     Scenario A: {text1[:80]}...")
            print(f"     Scenario B: {text2[:80]}...")
    else:
        print(f"\n✓ No high similarity pairs found (all < 0.60)")
    
    # Show lowest similarity pairs for context
    print(f"\n📊 TOP 5 MOST DIFFERENT PAIRS (GOOD DIVERSITY):")
    for i, (idx1, idx2, sim, text1, text2) in enumerate(results['low_sim_pairs'][:5], 1):
        print(f"\n  {i}. Similarity: {sim:.4f}")
        print(f"     Scenario A: {text1[:80]}...")
        print(f"     Scenario B: {text2[:80]}...")

print(f"\n{'='*80}\n")

In [ ]:
# Summary visualization and recommendations
print("=== SUMMARY: MODE COLLAPSE DETECTION ===\n")

# Summary statistics table
summary_data = []
for principle in HUMANE_PRINCIPLES:
    if principle in within_principle_results:
        stats = within_principle_results[principle]
        summary_data.append({
            'Principle': principle,
            'Min': f"{stats['min']:.3f}",
            'Mean': f"{stats['mean']:.3f}",
            'Max': f"{stats['max']:.3f}",
            'Std': f"{stats['std']:.3f}",
            'High Sim Pairs (≥0.60)': stats['n_high_sim']
        })

summary_df = pd.DataFrame(summary_data)
print(summary_df.to_string(index=False))

# Visualizations
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Box plot of similarity distributions
similarity_dists = []
labels = []
for principle in HUMANE_PRINCIPLES:
    if principle in within_principle_results:
        similarity_dists.append(within_principle_results[principle]['similarities'])
        labels.append(principle.replace('-', '\n'))

bp = ax1.boxplot(similarity_dists, labels=labels, patch_artist=True)
for patch in bp['boxes']:
    patch.set_facecolor('lightblue')

ax1.axhline(0.60, color='red', linestyle='--', alpha=0.7, label='Threshold (0.60)')
ax1.set_ylabel('Cosine Similarity')
ax1.set_title('Within-Principle Similarity Distributions', fontweight='bold', fontsize=12)
ax1.set_xticklabels(labels, rotation=45, ha='right', fontsize=9)
ax1.legend()
ax1.grid(True, alpha=0.3, axis='y')

# Bar chart of high similarity pairs
high_sim_counts = [within_principle_results[p]['n_high_sim'] for p in HUMANE_PRINCIPLES if p in within_principle_results]
principle_labels = [p.replace('-', '\n') for p in HUMANE_PRINCIPLES if p in within_principle_results]

colors = ['red' if count > 10 else 'orange' if count > 5 else 'green' for count in high_sim_counts]
bars = ax2.bar(principle_labels, high_sim_counts, color=colors, alpha=0.7)
ax2.set_ylabel('Number of High Similarity Pairs')
ax2.set_title('High Similarity Pairs (≥0.60) by Principle', fontweight='bold', fontsize=12)
ax2.set_xticklabels(principle_labels, rotation=45, ha='right', fontsize=9)
ax2.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'{int(height)}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

# Recommendations
print("\n=== RECOMMENDATIONS ===\n")
total_high_sim = sum(high_sim_counts)
if total_high_sim == 0:
    print("✓ No concerning high similarities detected within any principle.")
    print("  Scenarios within each principle appear sufficiently heterogeneous.")
else:
    print(f"⚠️  Found {total_high_sim} high similarity pairs (≥0.60) across all principles.\n")
    print("Principles requiring attention:")
    for principle, count in zip(HUMANE_PRINCIPLES, high_sim_counts):
        if count > 0:
            print(f"  • {principle}: {count} high-similarity pairs")
            if count > 10:
                print(f"    → Consider generating more diverse scenarios for this principle")
    print("\nReview the detailed results above to identify specific similar scenario pairs.")

print("\n✓ Mode collapse detection complete!")

## 3. Stratified Random Sampling for Manual Review

Generate a stratified random sample ensuring representation across all principles and domains.

**Sample Size:** ~10-15% of dataset (100-150 scenarios) provides 95% confidence with ~10% margin of error.

In [ ]:
def stratified_sample(df: pd.DataFrame, sample_size: int = 120, random_state: int = 42) -> pd.DataFrame:
    """
    Generate stratified random sample ensuring representation across principles.
    
    Args:
        df: Full dataset
        sample_size: Target sample size (default: 120, ~11.5% of 1039)
        random_state: Random seed for reproducibility
    
    Returns:
        Stratified sample DataFrame
    """
    # Calculate samples per principle (proportional stratified sampling)
    principle_counts = df['principle'].value_counts()
    
    samples_per_principle = {}
    for principle, count in principle_counts.items():
        proportion = count / len(df)
        n_samples = max(int(sample_size * proportion), 1)  # At least 1 per principle
        samples_per_principle[principle] = min(n_samples, count)  # Can't sample more than available
    
    # Adjust to hit target sample size
    total_samples = sum(samples_per_principle.values())
    if total_samples < sample_size:
        # Distribute remaining samples proportionally
        remaining = sample_size - total_samples
        sorted_principles = sorted(principle_counts.items(), key=lambda x: x[1], reverse=True)
        for principle, count in sorted_principles:
            if remaining <= 0:
                break
            if samples_per_principle[principle] < count:
                samples_per_principle[principle] += 1
                remaining -= 1
    
    print("=== STRATIFIED SAMPLING PLAN ===")
    print(f"Target sample size: {sample_size}")
    print(f"\nSamples per principle:")
    for principle in HUMANE_PRINCIPLES:
        if principle in samples_per_principle:
            n = samples_per_principle[principle]
            total = principle_counts[principle]
            pct = n / total * 100
            print(f"  {principle:40s} {n:3d} / {total:4d} ({pct:5.1f}%)")
    
    # Perform stratified sampling
    sample_dfs = []
    for principle, n_samples in samples_per_principle.items():
        principle_df = df[df['principle'] == principle]
        sample_df = principle_df.sample(n=n_samples, random_state=random_state)
        sample_dfs.append(sample_df)
    
    sample = pd.concat(sample_dfs, ignore_index=True)
    
    # Shuffle the sample
    sample = sample.sample(frac=1, random_state=random_state).reset_index(drop=True)
    
    print(f"\n✓ Generated stratified sample of {len(sample)} scenarios")
    
    return sample

# Generate sample (adjust sample_size as needed)
SAMPLE_SIZE = 120  # ~11.5% of 1039 scenarios
sample_df = stratified_sample(df, sample_size=SAMPLE_SIZE)

In [ ]:
# Verify sample distribution
print("\n=== SAMPLE DISTRIBUTION VERIFICATION ===")

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Compare principle distribution
full_dist = df['principle'].value_counts(normalize=True).sort_index() * 100
sample_dist = sample_df['principle'].value_counts(normalize=True).sort_index() * 100

comparison_df = pd.DataFrame({
    'Full Dataset': full_dist,
    'Sample': sample_dist
})

comparison_df.plot(kind='bar', ax=axes[0], color=['steelblue', 'coral'])
axes[0].set_title('Principle Distribution: Full Dataset vs Sample', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Principle')
axes[0].set_ylabel('Percentage (%)')
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=45, ha='right')
axes[0].legend()

# Compare domain distribution (top 10)
full_domain_dist = df['domain'].value_counts(normalize=True).head(10) * 100
sample_domain_dist = sample_df['domain'].value_counts(normalize=True).head(10) * 100

domain_comparison_df = pd.DataFrame({
    'Full Dataset': full_domain_dist,
    'Sample': sample_domain_dist
}).fillna(0)

domain_comparison_df.plot(kind='bar', ax=axes[1], color=['mediumseagreen', 'lightcoral'])
axes[1].set_title('Domain Distribution: Full Dataset vs Sample (Top 10)', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Domain')
axes[1].set_ylabel('Percentage (%)')
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=45, ha='right')
axes[1].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Export sample for manual review
review_columns = ['id', 'input', 'target', 'principle', 'domain', 'vulnerable_population', 'input_length']
review_sample = sample_df[review_columns].copy()

# Add columns for reviewer ratings
# Reviewers will code each scenario on multiple criteria
review_sample['reviewer_1_clear'] = ''  # Is input clear? (yes/no)
review_sample['reviewer_1_realistic'] = ''  # Is scenario realistic? (yes/no)
review_sample['reviewer_1_tests_principle'] = ''  # Does it test target principle? (yes/no)
review_sample['reviewer_1_quality'] = ''  # Overall quality (low/medium/high)
review_sample['reviewer_1_notes'] = ''  # Optional notes

review_sample['reviewer_2_clear'] = ''
review_sample['reviewer_2_realistic'] = ''
review_sample['reviewer_2_tests_principle'] = ''
review_sample['reviewer_2_quality'] = ''
review_sample['reviewer_2_notes'] = ''

review_sample['reviewer_3_clear'] = ''
review_sample['reviewer_3_realistic'] = ''
review_sample['reviewer_3_tests_principle'] = ''
review_sample['reviewer_3_quality'] = ''
review_sample['reviewer_3_notes'] = ''

# Save to CSV
output_file = 'review_sample.csv'
review_sample.to_csv(output_file, index=False)

print(f"✓ Exported {len(review_sample)} scenarios to '{output_file}'")
print(f"\nReview instructions:")
print("1. Each reviewer should independently fill in their columns")
print("2. For yes/no fields: use 'yes' or 'no'")
print("3. For quality: use 'low', 'medium', or 'high'")
print("4. Notes are optional but helpful for discussing disagreements")
print("5. After all reviews are complete, return to this notebook for inter-rater reliability analysis")

## 4. Inter-Rater Reliability Analysis

After manual reviews are completed, run this section to calculate **Fleiss' Kappa** - a statistical measure of inter-rater agreement.

**Interpretation (Landis & Koch criteria):**
- < 0.00: Poor agreement
- 0.00-0.20: Slight agreement
- 0.21-0.40: Fair agreement
- 0.41-0.60: Moderate agreement
- 0.61-0.80: Substantial agreement
- 0.81-1.00: Almost perfect agreement

In [ ]:
def fleiss_kappa(ratings_matrix: np.ndarray, categories: int) -> float:
    """
    Calculate Fleiss' Kappa for inter-rater reliability.
    
    Args:
        ratings_matrix: Matrix of shape (n_items, n_categories) where each cell
                       contains the number of raters who assigned that category to that item
        categories: Number of possible categories
    
    Returns:
        Fleiss' Kappa coefficient
    """
    n_items, n_categories = ratings_matrix.shape
    n_raters = ratings_matrix.sum(axis=1)[0]  # Total raters per item (should be constant)
    
    # Calculate P_i (proportion of agreement for each item)
    P_i = (np.sum(ratings_matrix ** 2, axis=1) - n_raters) / (n_raters * (n_raters - 1))
    P_bar = np.mean(P_i)  # Mean proportion of agreement
    
    # Calculate P_j (proportion of all assignments to category j)
    P_j = np.sum(ratings_matrix, axis=0) / (n_items * n_raters)
    P_e_bar = np.sum(P_j ** 2)  # Expected proportion of agreement by chance
    
    # Calculate Fleiss' Kappa
    if P_e_bar == 1.0:
        return 1.0  # Perfect agreement
    
    kappa = (P_bar - P_e_bar) / (1 - P_e_bar)
    return kappa


def interpret_kappa(kappa: float) -> str:
    """Interpret Kappa value using Landis & Koch criteria."""
    if kappa < 0.0:
        return "Poor"
    elif kappa < 0.20:
        return "Slight"
    elif kappa < 0.40:
        return "Fair"
    elif kappa < 0.60:
        return "Moderate"
    elif kappa < 0.80:
        return "Substantial"
    else:
        return "Almost Perfect"


def analyze_inter_rater_reliability(review_file: str = 'review_sample.csv') -> Dict:
    """
    Analyze inter-rater reliability from completed review file.
    
    Args:
        review_file: Path to CSV file with completed reviews
    
    Returns:
        Dictionary with Kappa scores and analysis
    """
    # Load completed reviews
    reviews_df = pd.read_csv(review_file)
    
    print("=== INTER-RATER RELIABILITY ANALYSIS ===")
    print(f"Loaded reviews for {len(reviews_df)} scenarios\n")
    
    results = {}
    
    # Analyze each criterion
    criteria = [
        ('clear', ['yes', 'no']),
        ('realistic', ['yes', 'no']),
        ('tests_principle', ['yes', 'no']),
        ('quality', ['low', 'medium', 'high'])
    ]
    
    for criterion, categories in criteria:
        print(f"\n--- {criterion.upper().replace('_', ' ')} ---")
        
        # Extract ratings from all reviewers
        reviewer_cols = [col for col in reviews_df.columns if criterion in col and 'notes' not in col]
        
        if not reviewer_cols:
            print(f"⚠ No data found for criterion '{criterion}'")
            continue
        
        # Build ratings matrix
        n_items = len(reviews_df)
        n_categories = len(categories)
        n_raters = len(reviewer_cols)
        
        ratings_matrix = np.zeros((n_items, n_categories))
        
        for item_idx in range(n_items):
            for reviewer_col in reviewer_cols:
                rating = str(reviews_df.loc[item_idx, reviewer_col]).strip().lower()
                if rating in categories:
                    cat_idx = categories.index(rating)
                    ratings_matrix[item_idx, cat_idx] += 1
        
        # Calculate Fleiss' Kappa
        kappa = fleiss_kappa(ratings_matrix, n_categories)
        interpretation = interpret_kappa(kappa)
        
        results[criterion] = {
            'kappa': kappa,
            'interpretation': interpretation,
            'n_raters': n_raters,
            'n_items': n_items
        }
        
        print(f"Fleiss' Kappa: {kappa:.3f} ({interpretation})")
        print(f"Raters: {n_raters}, Items: {n_items}")
        
        # Show category distribution
        category_totals = ratings_matrix.sum(axis=0)
        print(f"\nCategory distribution:")
        for cat, total in zip(categories, category_totals):
            pct = total / (n_items * n_raters) * 100
            print(f"  {cat:15s} {int(total):4d} ({pct:5.1f}%)")
    
    # Identify disagreements
    print("\n\n=== DISAGREEMENT ANALYSIS ===")
    
    for criterion, _ in criteria:
        reviewer_cols = [col for col in reviews_df.columns if criterion in col and 'notes' not in col]
        if len(reviewer_cols) < 2:
            continue
        
        # Find rows where reviewers disagree
        disagreements = []
        for idx, row in reviews_df.iterrows():
            ratings = [str(row[col]).strip().lower() for col in reviewer_cols]
            if len(set(ratings)) > 1:  # Not all the same
                disagreements.append({
                    'id': row['id'],
                    'input': row['input'][:80] + '...' if len(row['input']) > 80 else row['input'],
                    'ratings': ratings
                })
        
        if disagreements:
            print(f"\n{criterion.upper().replace('_', ' ')}: {len(disagreements)} disagreements ({len(disagreements)/len(reviews_df)*100:.1f}%)")
            print(f"Examples (showing first 5):")
            for disagreement in disagreements[:5]:
                print(f"  • {disagreement['id']}: {disagreement['ratings']}")
                print(f"    '{disagreement['input']}'")
    
    return results


# Run inter-rater reliability analysis
# NOTE: This will only work after reviews are completed and the CSV is filled in
try:
    reliability_results = analyze_inter_rater_reliability('review_sample.csv')
    
    # Visualize Kappa scores
    if reliability_results:
        criteria_names = []
        kappa_values = []
        
        for criterion, results in reliability_results.items():
            criteria_names.append(criterion.replace('_', ' ').title())
            kappa_values.append(results['kappa'])
        
        # Plot Kappa scores
        fig, ax = plt.subplots(figsize=(10, 6))
        bars = ax.bar(criteria_names, kappa_values, color='steelblue')
        
        # Color bars by interpretation
        for bar, kappa in zip(bars, kappa_values):
            if kappa < 0.40:
                bar.set_color('indianred')
            elif kappa < 0.60:
                bar.set_color('orange')
            elif kappa < 0.80:
                bar.set_color('gold')
            else:
                bar.set_color('mediumseagreen')
        
        # Add reference lines
        ax.axhline(0.40, color='red', linestyle='--', alpha=0.5, label='Fair/Moderate threshold')
        ax.axhline(0.60, color='orange', linestyle='--', alpha=0.5, label='Moderate/Substantial threshold')
        ax.axhline(0.80, color='green', linestyle='--', alpha=0.5, label='Substantial/Almost Perfect threshold')
        
        ax.set_title("Fleiss' Kappa Scores by Criterion", fontsize=14, fontweight='bold')
        ax.set_xlabel('Criterion')
        ax.set_ylabel("Fleiss' Kappa")
        ax.set_ylim(0, 1)
        ax.legend()
        plt.xticks(rotation=45, ha='right')
        plt.tight_layout()
        plt.show()

except FileNotFoundError:
    print("⚠ Review file not found. Please complete manual reviews first.")
except Exception as e:
    print(f"⚠ Could not analyze inter-rater reliability: {e}")
    print("This is expected if reviews haven't been completed yet.")

## 5. Recommendations & Next Steps

Based on the analysis above, document your findings and recommendations:

1. **Distribution Balance:** Are principles and domains evenly represented?
2. **Inter-Rater Reliability:** What is the level of agreement among reviewers?
3. **Quality Issues:** What common issues were identified during manual review?
4. **Action Items:** What changes should be made to improve the dataset?

Use the cells below to document your findings.

In [ ]:
# Document findings here
print("=== QA FINDINGS & RECOMMENDATIONS ===")
print("\n[Add your findings and recommendations after completing the analysis]")
print("\n1. Distribution Balance:")
print("   -")
print("\n2. Inter-Rater Reliability:")
print("   -")
print("\n3. Quality Issues:")
print("   -")
print("\n4. Action Items:")
print("   -")